In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Houma,US,2021-03-25 12:41:28,29.5958,-90.7195,73.99,94,90,12.66
1,1,Kampot,KH,2021-03-25 12:41:29,10.6167,104.1833,84.20,78,0,2.30
2,2,Tasiilaq,GL,2021-03-25 12:41:29,65.6145,-37.6368,3.20,72,40,2.30
3,3,Saint-Leu,RE,2021-03-25 12:41:30,-21.1500,55.2833,78.80,89,75,8.05
4,4,Rikitea,PF,2021-03-25 12:41:30,-23.1203,-134.9692,79.63,72,62,17.07
...,...,...,...,...,...,...,...,...,...,...
551,551,Ingham,US,2021-03-25 12:45:57,42.6000,-84.3833,46.00,81,1,7.81
552,552,Half Moon Bay,US,2021-03-25 12:45:58,37.4636,-122.4286,51.01,93,90,19.57
553,553,Leshukonskoye,RU,2021-03-25 12:45:58,64.8989,45.7622,36.73,79,100,13.60
554,554,Bar Harbor,US,2021-03-25 12:45:59,44.3876,-68.2039,46.99,100,90,6.71


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Kampot,KH,2021-03-25 12:41:29,10.6167,104.1833,84.20,78,0,2.30
3,3,Saint-Leu,RE,2021-03-25 12:41:30,-21.1500,55.2833,78.80,89,75,8.05
4,4,Rikitea,PF,2021-03-25 12:41:30,-23.1203,-134.9692,79.63,72,62,17.07
8,8,Cururupu,BR,2021-03-25 12:41:32,-1.8283,-44.8683,75.00,96,100,1.48
10,10,Farafangana,MG,2021-03-25 12:41:32,-22.8167,47.8333,78.44,80,46,8.88
15,15,Baculin,PH,2021-03-25 12:41:35,7.4525,126.5842,78.22,84,22,4.72
19,19,Matara,LK,2021-03-25 12:41:36,5.9485,80.5353,75.34,88,100,6.93
22,22,Mahebourg,MU,2021-03-25 12:41:37,-20.4081,57.7000,80.60,83,75,3.44
35,35,Saint-Philippe,RE,2021-03-25 12:41:43,-21.3585,55.7679,78.80,89,75,8.05
39,39,Yulara,AU,2021-03-25 12:41:44,-25.2406,130.9889,80.60,34,0,5.75


In [30]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kampot,KH,84.20,10.6167,104.1833,
3,Saint-Leu,RE,78.80,-21.1500,55.2833,
4,Rikitea,PF,79.63,-23.1203,-134.9692,
8,Cururupu,BR,75.00,-1.8283,-44.8683,
10,Farafangana,MG,78.44,-22.8167,47.8333,
15,Baculin,PH,78.22,7.4525,126.5842,
19,Matara,LK,75.34,5.9485,80.5353,
22,Mahebourg,MU,80.60,-20.4081,57.7000,
35,Saint-Philippe,RE,78.80,-21.3585,55.7679,
39,Yulara,AU,80.60,-25.2406,130.9889,


In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [32]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kampot,KH,84.20,10.6167,104.1833,Two Moons Hotel
3,Saint-Leu,RE,78.80,-21.1500,55.2833,La Caz Oceane
4,Rikitea,PF,79.63,-23.1203,-134.9692,Pension Maro'i
8,Cururupu,BR,75.00,-1.8283,-44.8683,Pousada Través'cia
10,Farafangana,MG,78.44,-22.8167,47.8333,Miarina
...,...,...,...,...,...,...
531,Mocuba,MZ,77.54,-16.8392,36.9856,Phama Lodge
542,Bah,IN,83.79,26.8700,78.5975,Chambal Wildlife Safari Lodge
545,Salalah,OM,84.20,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
549,Vanimo,PG,79.02,-2.6741,141.3028,Vahmoneh Lodge Vanimo


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [28]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))